# 概要

Juliaとawabi.jlによる形態素解の実験

In [18]:
# ライブラリをインポート
using HTTP, JSON, CSV, TableOperations, DataFrames

# ニコニコ動画のデータを収集する関数を定義
function collect_nico_data(ranking_type, date, file_type, endpoint="https://dcdn.cdn.nimg.jp/nicovideo/old-ranking")
    response = HTTP.get("$endpoint/$ranking_type/$date/$file_type")                      # リクエストを送信
    response.status != 200 && println("APIにアクセスできませんでした。")                 # エラーメッセージを表示
    data = JSON.parse(String(response.body))                                             # レスポンスをJSON形式に変換
    CSV.write("$(file_type).csv", data; transform=((col, val)->something(val, missing))) # データをCSVファイルに保存（transformオプションを追加）
end

# 関数を呼び出してデータを収集する（例）
collect_nico_data("daily", "2023-09-23", "file_name_list.json")

"file_name_list.json.csv"

In [20]:
function search_tag(tag, file_name)
  file_name_list = CSV.read(file_name, DataFrame)
  # FIXME: ここでtagが見つからなかった場合の処理を追加する
  return file_name_list[coalesce.(file_name_list.tag, "") .== "アーマードコア6", :].file
end
ac6_file_name = search_tag("アーマードコア6", "file_name_list.json.csv")

1-element Vector{String31}:
 "game_04.json"

In [5]:
using DataFrames